In [ ]:
import torch 
import torchvision 
import numpy as np
import pandas as pd
from torchvision.models import video
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch import optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm 

np.random.seed(42)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
!ls ../input/haar-cascade-drowsiness-dataset-fold1-1

In [ ]:
data = np.load('../input/haar-cascade-drowsiness-dataset-fold1-1/data_3k.npy')
label = np.load('../input/haar-cascade-drowsiness-dataset-fold1-1/label_3k.npy')
test = np.load('../input/haar-cascade-drowsiness-dataset-fold1-1/test_500.npy')
test_label = np.load('../input/haar-cascade-drowsiness-dataset-fold1-1/test_label_500.npy')
# ../input/haar-cascade-drowsiness-dataset-fold1-1

In [ ]:
model = video.r3d_18(pretrained=True)

In [ ]:
data = torch.Tensor(data) / 255.0
label = torch.Tensor(label) / 5.0
test = torch.Tensor(test) / 255.0
test_label = torch.Tensor(test_label) / 5.0

In [ ]:
data.shape, test.shape

In [ ]:
data = data.permute((0,4,1,2,3))
test = test.permute(((0,4,1,2,3)))
label = label.long()
test_label = test_label.long()

In [ ]:
model.fc = nn.Linear(512, 3)
for param in model.parameters() :
    param.require_grad = False
model.fc.weight.require_grad = True
model.fc.bias.require_grad = True 

model = model.to(device)

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2, last_epoch=-1) 

In [ ]:
class TrainDataset(Dataset) :
    def __len__(self) :
        return len(data)
    
    def __getitem__(self, idx) :
        data_batch = data[idx]
        label_batch = label[idx]
        return data_batch, label_batch
class TestDataset(Dataset) :
    def __len__(self) :
        return len(test)
    
    def __getitem__(self, idx) :
        data_batch = test[idx]
        label_batch = test_label[idx]
        return data_batch, label_batch

In [ ]:
train_dataset = TrainDataset()
train_loader = DataLoader(train_dataset, batch_size=20)

test_dataset = TestDataset()
test_loader = DataLoader(test_dataset, batch_size=20)

In [ ]:
epochs = 10
loss_train_epoch = []
loss_test_epoch = []
for e in range(epochs) :
    loss_train = 0.0
    loss_test = 0.0
    count_train_batch = 0
    count_test_batch = 0
    for local_batch, local_labels in tqdm(train_loader) :
        model.train()
        optimiser.zero_grad()
        local_batch = local_batch.to(device)
        local_labels = local_labels.to(device)
        predictions = model(local_batch)
        predictions = torch.softmax(predictions, dim=1)
#         print(predictions)
        loss_val = loss(predictions, local_labels)
        loss_val.backward()
        optimiser.step()
        count_train_batch += 1
        with torch.set_grad_enabled(False) :
            loss_train += loss_val.cpu().item()
    
    with torch.set_grad_enabled(False) :
        for local_batch, local_labels in tqdm(test_loader) :
            local_batch = local_batch.to(device)
            local_labels = local_labels.to(device)
            predictions = model(local_batch)
            loss_val = loss(predictions, local_labels)
            count_test_batch += 1
            loss_test += loss_val.cpu().item()
        loss_test /= count_test_batch
        loss_train /= count_train_batch
        loss_train_epoch.append(loss_train)
        loss_test_epoch.append(loss_test)

In [ ]:
plt.plot(range(epochs), loss_train_epoch)
plt.legend(['train_loss'])
plt.show()

plt.plot(range(epochs), loss_test_epoch)
plt.legend(['test_loss'])
plt.show()

In [ ]:

optimiser2 = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

with torch.set_grad_enabled(False) :
    predictions = model(test.to(device).cpu()
    print(classification_report(predictions.argmax(dim=1), y_test))
    

In [ ]:
with torch.set_grad_enabled(False) :
    predictions = model(x_train[:100].permute((0,4,1,2,3)).to(device)).cpu()
    print(classification_report(predictions.argmax(dim=1), y_train[:100]))


In [ ]:
torch.save(model, 'R3D_18.pt')